In [ ]:
################################## LENDO A BASE OPERACIONAL DA SEMANA PARA HIGIENIZAR
import pandas as pd
import numpy as np
import pysftp
import io
import math

# CARREGANDO APENAS AS COLUNAS NECESSARIAS BGPRA HIGIENIZAR (CPF,NOME,TEL,EMAIL)
colunas_necessarias_para_higienizar = ['CPF','NOME_x','TELEFONE','EMAIL','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO','BASE','ATRASO']

telecob_hugo_vigente = pd.read_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO/BASE_OPERACIONAL_TELECOB_HUGO.txt",sep=';', usecols= colunas_necessarias_para_higienizar,dtype=str)


### colocando a base vertical e concatenando
df_1_contratante = telecob_hugo_vigente.loc[:,['CPF','NOME_x','TELEFONE','EMAIL','ATRASO','BASE']]
df_2_higienizados = telecob_hugo_vigente.loc[:,['CPF','NOME_x','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO','ATRASO','BASE']]
df_2_higienizados = df_2_higienizados.rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})
df_2_higienizados = df_2_higienizados.rename(columns={'EMAIL_HIGIENIZADO':'EMAIL'})

telecob_hugo_vigente = pd.concat([df_1_contratante,df_2_higienizados])
telecob_hugo_vigente = telecob_hugo_vigente.drop_duplicates(['CPF', 'NOME_x','TELEFONE', 'EMAIL'])
telecob_hugo_vigente['ATRASO'] = telecob_hugo_vigente['ATRASO'].astype('int32')

########### ORGANIZANDO OS CPF'S E RESETANDO O INDEX PRA BAIXAR OS ARQUIVOS
telecob_hugo_vigente = telecob_hugo_vigente.sort_values(['ATRASO','CPF'])
telecob_hugo_vigente = telecob_hugo_vigente.reset_index(drop=True)

In [ ]:
#### AVALIAÇÃO PARA SABER OQ VAI HIGIENIZAR
telecob_hugo_vigente.query('BASE == "Claro Fixo"').drop_duplicates(['CPF'])


In [ ]:
###### FILTRANDO CARTEIRAS PELAS SEGMENTAÇÕES DE ATRASO QUE VAMOS HIGIENIZAR 
### CARTEIRAS CLARO 
higi_movel = telecob_hugo_vigente.query('BASE == "Claro Móvel"  and ATRASO >= 0  and ATRASO <= 1850  ')
higi_tv = telecob_hugo_vigente.query('BASE == "Claro TV"  and ATRASO >= 0  and ATRASO <= 3500 ')
higi_fixo = telecob_hugo_vigente.query('BASE == "Claro Fixo"')


#### CARTEIRAS NET E GEVENUE
higi_gevenue = telecob_hugo_vigente.query('BASE == "GEVENUE" and ATRASO >= 0   and ATRASO <= 4000')
# higi_net = telecob_hugo_vigente.query('BASE == "NET" and ATRASO <= 99999 ')

In [ ]:
# telecob_hugo_vigente = pd.concat([higi_movel,higi_tv,higi_fixo,higi_gevenue,higi_net],ignore_index=True).drop_duplicates(['CPF','NOME_x','TELEFONE','EMAIL'])
telecob_hugo_vigente = pd.concat([higi_movel,higi_tv,higi_fixo,higi_gevenue],ignore_index=True).drop_duplicates(['CPF','NOME_x','TELEFONE','EMAIL'])

### SEPARANDO UM DATAFRAME PARA GERAR UM QUADRO DO QUE JÁ HIGIENIZAMOS
base_enviada_para_higienizar = telecob_hugo_vigente
### PREPARANDO O LAYOUT FINAL PARA ENVIAR VIA SFTP
telecob_hugo_vigente = telecob_hugo_vigente.loc[:,['CPF','NOME_x','TELEFONE','EMAIL']]
telecob_hugo_vigente

In [ ]:
# estabelecer conexão SFTP
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None   # ignore host key verification
sftp = pysftp.Connection(host='sftp-afinder.koiketec.com', username='user1009', password='8GEeNgVW5DgbmYyp',port=17592, cnopts=cnopts)


# Definir o tamanho desejado para cada lote

tamanho_lote = math.ceil(len(telecob_hugo_vigente) / 3)

# Calcular o número de lotes necessário
num_lotes = int(np.ceil(len(telecob_hugo_vigente) / tamanho_lote))

# Dividir o DataFrame em lotes
lotes = np.array_split(telecob_hugo_vigente, num_lotes)

# Verificar o tamanho de cada lote e ajustar se necessário
for i, lote in enumerate(lotes):
    if len(lote) > tamanho_lote:
        # Dividir o lote em sublotes menores
        sublotes = np.array_split(lote, len(lote) // tamanho_lote + 1)
        # Substituir o lote original pelos sublotes
        lotes[i:i+1] = sublotes

# Enviar os lotes via SFTP
for i, lote in enumerate(lotes):
    filename = f'CLAR_NET_GEV_lote_{i}_COBRANCA.csv'
    with io.StringIO() as sftp_file:
        lote.to_csv(sftp_file, sep=';', index=False)
        sftp.putfo(io.BytesIO(sftp_file.getvalue().encode()), f'/ftp/Entrada/{filename}')

# Fechar a conexão com o servidor
sftp.close()

In [ ]:
base_enviada_para_higienizar = base_enviada_para_higienizar.drop_duplicates(['CPF'])
base_enviada_para_higienizar = base_enviada_para_higienizar.groupby(['BASE'])['CPF'].count().reset_index()
 

def obter_linhas(df):
    primeira_linha_base = df['BASE'].iloc[0]
    primeira_linha_atraso = df['ATRASO'].iloc[0]
    ultima_linha_atraso = df['ATRASO'].iloc[-1]
    return primeira_linha_base, primeira_linha_atraso, ultima_linha_atraso

# Retorno Relatório Móvel
primeira_linha_base_movel, primeira_linha_atraso_movel, ultima_linha_atraso_movel = obter_linhas(higi_movel)

# Retorno Relatório TV
primeira_linha_base_tv, primeira_linha_atraso_tv, ultima_linha_atraso_tv = obter_linhas(higi_tv)

# Retorno Relatório Fixo
primeira_linha_base_fixo, primeira_linha_atraso_fixo, ultima_linha_atraso_fixo = obter_linhas(higi_fixo)

# Retorno Relatório Gevenue
primeira_linha_base_gevenue, primeira_linha_atraso_gevenue, ultima_linha_atraso_gevenue = obter_linhas(higi_gevenue)

# Retorno Relatório Net
# primeira_linha_base_net, primeira_linha_atraso_net, ultima_linha_atraso_net = obter_linhas(higi_net)


def criar_dataframe_retorno(carteira, menor_atraso, maior_atraso):
    dados = {
        'CARTEIRA': [carteira],
        'MENOR_ATRASO': [menor_atraso],
        'MAIOR_ATRASO': [maior_atraso]
    }
    return pd.DataFrame(dados)

# Retorno Relatório Móvel
higi_semana_movel = criar_dataframe_retorno(primeira_linha_base_movel, primeira_linha_atraso_movel, ultima_linha_atraso_movel)

# Retorno Relatório TV
higi_semana_tv = criar_dataframe_retorno(primeira_linha_base_tv, primeira_linha_atraso_tv, ultima_linha_atraso_tv)

# Retorno Relatório Fixo
higi_semana_fixo = criar_dataframe_retorno(primeira_linha_base_fixo, primeira_linha_atraso_fixo, ultima_linha_atraso_fixo)

# Retorno Relatório Gevenue
higi_semana_gevenue = criar_dataframe_retorno(primeira_linha_base_gevenue, primeira_linha_atraso_gevenue, ultima_linha_atraso_gevenue)

# Retorno Relatório Net
# higi_semana_net = criar_dataframe_retorno(primeira_linha_base_net, primeira_linha_atraso_net, ultima_linha_atraso_net)

# Concatenar todos os DataFrames de retorno
# higienizacao_semana = pd.concat([higi_semana_movel, higi_semana_tv, higi_semana_fixo, higi_semana_gevenue, higi_semana_net], ignore_index=True)
higienizacao_semana = pd.concat([higi_semana_movel, higi_semana_tv, higi_semana_fixo, higi_semana_gevenue], ignore_index=True)
hienizadao_semana_final = pd.merge(base_enviada_para_higienizar, higienizacao_semana, right_on=["CARTEIRA"], left_on=["BASE"], how="inner")

hienizadao_semana_final = hienizadao_semana_final.loc[:,['CARTEIRA','CPF',	'MENOR_ATRASO',	'MAIOR_ATRASO']]

In [ ]:
hienizadao_semana_final

In [ ]:
hienizadao_semana_final.to_excel(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\FEEDBACK_HIGIENIZAÇÃO\NOVEMBRO_2023/HIGIENIZAÇÃO_CLARO_NET_GEVENUE_0711.xlsx",index=False)